In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import date, timedelta, datetime
import re
import concurrent.futures

In [ ]:
# Setting variables to be used below
maxTweets = 10

search_phrases = ["CovidVaccine",
                  "COVIDVaccine",
                  "COVIDVaccines",
                  "vaccinedreams",
                  "GetVaccinated",
                  "vaccinate",
                  "vaccines",
                  "COVID19Vaccine",
                  "vaccinated",
                  "VaccineApartheid",
                  "vaccine",
                  "Vaccine",
                  "VaccinateAll58",
                  "CovidVaccines",
                  "COVID19vaccine",
                  "VaccineDreams",
                  "vaccinehesitancy",
                  "GetVaccinatedNow",
                  "modernavaccine",
                  "FullyVaccinated",
                  "COVIDvaccines",
                  "EndVaccineApartheid",
                  "VaccinesWork",
                  "VaccineConfidence",
                  "COVID19Vaccines",
                  "COVIDVaccinesSaveLives",
                  "vaccinedeliberation",
                  "VaccinateALL58",
                  "freethevaccine",
                  "COVID19Vaccination",
                  "Vaccin8",
                  "COVIDvaccine",
                  "VaccinesSaveLives",
                  "getvaccinated",
                  "PleaseGetVaccinated",
                  "Vaccines",
                  "vaccinepassports",
                  "vaccinatedconnections",
                  "coronavirusvaccine",
                  "ModernaVaccine",
                  "TakeTheVaccine",
                  "vaccination",
                  "VaccineEquityNow",
                  "VaccinEquity",
                  "VaccinatetheWorld",
                  "vaccinesforall",
                  "thepeoplesvaccine",
                  "vaccinatetheworld",
                  "vaccineequity",
                  "mRNAvaccines",
                  "ivaccinatebecause",
                  "Covidvaccine",
                  "VaccineEquity",
                  "COVIDVaccination"]

In [ ]:
tweets_list = []
tweet_id_list = []

def twitter(phrase):
    startdate = date.today() - timedelta(days = 766)
    while True:
        enddate = (startdate + timedelta(days=1))
        if enddate == date.today():
            break
        x = 0
        for tweet in sntwitter.TwitterSearchScraper(phrase + ' since:' + str(startdate) + ' until:' + str(enddate)).get_items():
            if x>=maxTweets:
                break
            if tweet.lang == "en":
                if tweet.coordinates == None:
                    tweet.coordinates = ""

                if tweet.place == None:
                    tweet.place =  ""

                tweet.user.location = re.sub(r'\s{2,}', '', tweet.user.location)

                if tweet.id not in tweet_id_list:
                    tweets_list.append([phrase,
                                        tweet.date,
                                        tweet.id,
                                        tweet.content,
                                        tweet.user.username,
                                        tweet.user.location,
                                        tweet.place,
                                        tweet.coordinates,
                                        tweet.user.verified,
                                        tweet.user.followersCount,
                                        tweet.retweetCount,
                                        tweet.lang])
                    tweet_id_list.append(tweet.id)
                    x += 1
        startdate += timedelta(days=1)


In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(twitter, search_phrases)

In [ ]:
# Creating list to append tweet data to
tweets_list_df = pd.DataFrame(tweets_list, columns =['Phrase',
                                                     'Date',
                                                     'Tweet ID',
                                                     'Content',
                                                     'Username',
                                                     'Location',
                                                     'Place',
                                                     'Coordinates',
                                                     'Verified',
                                                     'FollowerCount',
                                                     'retweetCount',
                                                     'Language'])

In [ ]:
tweets_list_df.head()

,Phrase,Date,Tweet ID,Content,Username,Location,Place,Coordinates,Verified,FollowerCount,retweetCount,Language
0,COVID19Vaccine,2020-03-01 22:18:28+00:00,1234241622046437376,@SenSchumer Of course. Anything else would be...,redirectloop,"Los Angeles, CA",,,False,2747,0,en
1,COVID19Vaccine,2020-03-01 12:27:16+00:00,1234092843431550985,"COVID-19 Vaccine Shipped, and Drug Trials Star...",nou2555,rPod Coworking Space,,,False,596,0,en
2,GetVaccinated,2020-03-01 18:15:12+00:00,1234180400798408704,Survey data from @CDCgov show #HPV #vaccinatio...,NFIDvaccines,"Bethesda, MD",,,True,8545,2,en
3,GetVaccinated,2020-03-01 00:27:13+00:00,1233911633774366722,"Hey, what’s in the box lady? \nGetting my seco...",RationalVaxxer,New Jersey,,,False,457,0,en
4,COVIDVaccine,2020-03-01 23:42:03+00:00,1234262655294066688,@BarnowskyLarry @BonginoReport @Strandjunker T...,Glory2G_d,1984,,,False,1432,0,en


In [ ]:
tweets_list_df.to_csv('tweet_output.csv')

In [ ]:
len(tweets_list_df)

222232